**Imports:**

In [ ]:
import graphistry
import igraph
import pandas as p

from py2neo import Graph, Node, Relationship, authenticate

**Authentication:**

In [ ]:
try:
    graphistry.register('$api_key$')
    print('\n✨ Graphistry API Connection - OK')
except Exception,e: print str(e)

try:
    host = '127.0.0.1:7474'
    url = 'http://127.0.0.1:7474/db/data/'
    username = 'neo4j'
    password = 'neo4j'
    
    authenticate(host, username, password)
    graph = Graph(url)
    print('✨ Neo4j Bolt Connection - OK')
except Exception,e: print str(e)

**Neo4j Cypher Query:**

In [ ]:
query = p.DataFrame(graph.data(
'''
MATCH (u1:User)-[:POSTS]->(t1:Tweet)<-[:RETWEETS]-(t2:Tweet)<-[:POSTS]-(u2:User)
RETURN u1.screen_name, u2.screen_name, t1.tid, t2.tid, t1.text, t2.text
LIMIT 200000
'''
))

type(query), query.shape

**Sample Data:**

In [ ]:
query.head(8)

**Bind to Graphistry:**

In [ ]:
g = graphistry.bind(source='u2.screen_name', destination='u1.screen_name', node='screen_name')

**Pandas to iGraph:**

In [ ]:
ig = g.pandas2igraph(query)

In [ ]:
ig.vs['pagerank'] = ig.pagerank(directed=False)
ig.vs['betweenness'] = ig.betweenness(directed=True)
ig.es['ebetweenness'] = ig.edge_betweenness(directed=True)

In [ ]:
uig = ig.copy()
uig.to_undirected()
ig.vs['community_infomap'] = uig.community_infomap().membership
ig.vs['community_louvain'] = uig.community_multilevel().membership
igraph.summary(ig)

**Send to Graphistry API:**

In [ ]:
g.bind(point_color='', point_size='').plot(ig)